#Домашнее задание 1

##Данные

In [1]:
import numpy as np
import pandas as pd

In [2]:
url1="http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df=pd.read_csv(url1, header=None, names =  
                     ("sepal.length","sepal.width","petal.length","petal.width",    
                     "species"))

In [3]:
df

,sepal.length,sepal.width,petal.length,petal.width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
set(df['species'])

{'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}

In [5]:
def make_labels(x, species):
    if x['species'] == species:
        return 1
    else:
        return -1

In [6]:
df['label_setosa'] = df.apply(lambda x: make_labels(x,'Iris-setosa'), axis=1)
df['label_versicolor'] = df.apply(lambda x: make_labels(x,'Iris-versicolor'), axis=1)
df['label_virginica'] = df.apply(lambda x: make_labels(x,'Iris-virginica'), axis=1)

In [7]:
df

,sepal.length,sepal.width,petal.length,petal.width,species,label_setosa,label_versicolor,label_virginica
0,5.1,3.5,1.4,0.2,Iris-setosa,1,-1,-1
1,4.9,3.0,1.4,0.2,Iris-setosa,1,-1,-1
2,4.7,3.2,1.3,0.2,Iris-setosa,1,-1,-1
3,4.6,3.1,1.5,0.2,Iris-setosa,1,-1,-1
4,5.0,3.6,1.4,0.2,Iris-setosa,1,-1,-1
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,-1,-1,1
146,6.3,2.5,5.0,1.9,Iris-virginica,-1,-1,1
147,6.5,3.0,5.2,2.0,Iris-virginica,-1,-1,1
148,6.2,3.4,5.4,2.3,Iris-virginica,-1,-1,1


Делим данные.

In [8]:
features_cols = ['sepal.length', 'sepal.width', 'petal.length', 'petal.width']

In [9]:
X = np.array(df[features_cols])
y_setosa = np.array(df['label_setosa'])
y_versicolor = np.array(df['label_versicolor'])
y_virginica = np.array(df['label_virginica'])

In [10]:
from sklearn.model_selection import train_test_split

X_train_setosa, X_test_setosa, y_train_setosa,  y_test_setosa = train_test_split(X, y_setosa, test_size=0.1, random_state=6)
X_train_versicolor, X_test_versicolor, y_train_versicolor, y_test_versicolor = train_test_split(X, y_versicolor, test_size=0.1, random_state=6)
X_train_virginica, X_test_virginica, y_train_virginica, y_test_virginica = train_test_split(X, y_virginica, test_size=0.1, random_state=6)

##Задание 1

Функции для расчёта $L_i(w)$(без регуляризации) и $\nabla L_i(w)$.

In [12]:
def logistic_regression_loss(x, y, weights):
    M = np.dot(x, weights) * y
    return np.log(1 + np.exp(-M)) 

def gradient_logistic_regression(x, y, weights, reg):
    M = np.dot(x, weights) * y
    exponent = np.exp(-M)

    return np.array([-y * x[t] * exponent / (1 + exponent) + 2 * reg * weights[t] for t in range(len(x))])


Стохастический градиентный спуск (исп. лекции К.В. Воронцова):

In [35]:
def SGD(X_data, y_data, loss=logistic_regression_loss, gradient=gradient_logistic_regression, h=1e-2, reg=0.5):
    eps = 1e-7
    np.random.seed(16)
    weights = np.random.random_sample(X_data.shape[1])
    
    Q = np.mean([loss(x, y, weights) for x, y in zip(X_data, y_data)]) + reg * np.sum(weights ** 2) 
    iter = 0

    while True:
        iter += 1

        index = np.random.random_integers(len(X_data)) - 1
        x = X_data[index]
        eps_x = loss(x, y_data[index], weights)
        grad_L = gradient(x, y_data[index], weights, reg)
        prev_weights = weights
        weights = weights - h * grad_L
        prev_Q = Q
        lam = 1 / iter
        Q = lam * eps_x + (1 - lam) * Q
        # print(f'index = {index}, Q = {Q}')

        if abs(prev_Q - Q) < eps or np.all(np.absolute(weights - prev_weights) < eps) or  iter == 30000:
            break

    return weights



Функция для классификации:

In [15]:
def logistic_regression_classify(X_data, weights):
    return np.sign([np.dot(x, weights) for x in X_data])

Функция для подбора параметров:

In [62]:
h_vars = [1e-2, 1e-3, 1e-4, 1e-5]
reg_vars = [0.1, 0.5, 1, 5]

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def count_test_accuracy(X_train_data, X_test_data, y_train_data, y_test_data, h, reg, algorithm):
    weights = SGD(X_train_data, y_train_data, h=h, reg=reg)
    return accuracy_score(y_test_data, logistic_regression_classify(X_test_data, weights))

def choose_params(X_train_data, X_test_data, y_train_data, y_test_data, algorithm=SGD):
    best_accuracy = 0.
    best_params = {'h': None, 'reg': None}
    for h in h_vars:
        for reg in reg_vars:
            accuracy_array=[]
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            for train_index, test_index in cv.split(y_train_data):
                X_train, X_test = X_train_data[train_index], X_train_data[test_index]
                y_train, y_test = y_train_data[train_index], y_train_data[test_index]
                weights = algorithm(X_train, y_train, h=h, reg=reg)
                acc = accuracy_score(y_test, logistic_regression_classify(X_test, weights))
                accuracy_array.append(acc)
            accuracy = np.mean(accuracy_array)

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {'h': h, 'reg': reg}

                if best_accuracy == 1.: 
                    return best_accuracy, count_test_accuracy(X_train_data, X_test_data, y_train_data, y_test_data, h, reg, algorithm), best_params
            
    return best_accuracy, count_test_accuracy(X_train_data, X_test_data, y_train_data, y_test_data, best_params['h'], best_params['reg'], algorithm), best_params
                    



Результаты (первое значение - accuracy на кроссвалидации, второе - accuracy на тестовой выборке, третье - оптимальные параметры).

In [42]:
choose_params(X_train_setosa, X_test_setosa, y_train_setosa, y_test_setosa)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 108 + 1) instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(1.0, 1.0, {'h': 0.01, 'reg': 0.1})

In [43]:
choose_params(X_train_versicolor, X_test_versicolor, y_train_versicolor, y_test_versicolor)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 108 + 1) instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(0.6592592592592592, 0.7333333333333333, {'h': 0.0001, 'reg': 0.1})

In [44]:
choose_params(X_train_virginica, X_test_virginica, y_train_virginica, y_test_virginica)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 108 + 1) instead
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(0.837037037037037, 0.8666666666666667, {'h': 0.001, 'reg': 0.1})

##Задание 2

Функция для расчёта $L_S(w)$:

In [39]:
def logistic_regression_loss_BGD(X_data, y_data, weights, reg):
    M_values = np.array([np.dot(x, weights) * y for x, y in zip(X_data, y_data)])
    return np.mean([np.log(1 + np.exp(-M)) for M in M_values]) + reg * np.sum(weights ** 2) 


Batch gradient descent:

In [67]:
def BGD(X_data, y_data, loss=logistic_regression_loss_BGD, gradient=gradient_logistic_regression, h=1e-4, reg=0.5):
    eps = 1e-7
    np.random.seed(16)
    weights = np.random.random_sample(X_data.shape[1])
    
    Q = loss(X_data, y_data, weights, reg)
    iter = 0
    batch_size = 18
    while True:
        iter += 1
        grads = []
        for i in range(0, len(y_data), batch_size):
            slice_batch = slice(i, min(i + batch_size, len(y_data)))
            X_batch = X_data[slice_batch]
            y_batch = y_data[slice_batch]
            grad_L = np.sum([gradient(x, y, weights, reg) for x, y in zip(X_batch, y_batch)], axis=0)
            grads.append(grad_L)
        grad_L = np.mean(grads)
        prev_weights = weights
        weights = weights - h * grad_L
        prev_Q = Q
        Q = loss(X_data, y_data, weights, reg)
        # print(f' Q = {Q}')
        if abs(prev_Q - Q) < eps or np.all(np.absolute(weights - prev_weights) < eps) or  iter == 5000:
            break
    # print(iter)
    return weights


Результаты (первое значение - accuracy на кроссвалидации, второе - accuracy на тестовой выборке, третье - оптимальные параметры).

In [68]:
choose_params(X_train_setosa, X_test_setosa, y_train_setosa, y_test_setosa, algorithm=BGD)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(0.6592592592592592, 1.0, {'h': 0.01, 'reg': 0.1})

In [69]:
choose_params(X_train_versicolor, X_test_versicolor, y_train_versicolor, y_test_versicolor, algorithm=BGD)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(0.6592592592592592, 0.5333333333333333, {'h': 0.001, 'reg': 0.1})

In [70]:
choose_params(X_train_virginica, X_test_virginica, y_train_virginica, y_test_virginica, algorithm=BGD)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: This function is deprecated. Please call randint(1, 135 + 1) instead
  if sys.path[0] == '':


(0.6814814814814816, 0.8666666666666667, {'h': 0.01, 'reg': 0.1})

##Задание 3

SGD:

+ Высокая скорость работы
+ Accuracy получилось лучше (на "versicolor") либо таким же (на "setosa" и "virginica")

BGD:

+ Не дал ни лучшей скорости, ни лучшего качества.

Стоит заметить, что выборка, на которой проводились эксперименты, крайне мала. Поэтому вряд ли данные результаты можно назвать репрезентативными.